In [1]:
import pandas as pd
from sklearn.metrics import r2_score, accuracy_score
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from rustrees.rustrees import DecisionTree, RandomForest
import rustrees.tree as rt
import time
import numpy as np

In [2]:
datasets = {
    "reg": ["diabetes", "housing", "dgp"],
    "clf": ["breast_cancer", "titanic"]
}

In [3]:
def evaluate_dataset(dataset, problem, model="dt", max_depth=5, n_estimators=10, n_repeats=100):
    df_train = pd.read_csv(f"../../datasets/{dataset}_train.csv")
    df_test = pd.read_csv(f"../../datasets/{dataset}_test.csv")
    
    df_train_rt = rt.from_pandas(df_train)
    df_test_rt = rt.from_pandas(df_test)

    if problem == "reg":
        metric_fn = r2_score
        metric = "r2"
        if model == "dt":
            model_sk = DecisionTreeRegressor(max_depth=max_depth)
        elif model == "rf":
            model_sk = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, n_jobs=-1)
    elif problem == "clf":
        metric_fn = accuracy_score
        metric = "acc"
        if model == "dt":
            model_sk = DecisionTreeClassifier(max_depth=max_depth)
        elif model == "rf":
            model_sk = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, n_jobs=-1)
    
    start_time = time.time()
    results_sk = []
    for _ in range(n_repeats):
        model_sk.fit(df_train.drop("target", axis=1), df_train.target)
        results_sk.append(metric_fn(df_test.target, model_sk.predict(df_test.drop("target", axis=1))))
    sk_time = (time.time() - start_time)/n_repeats
    sk_mean = np.mean(results_sk)
    sk_std = np.std(results_sk)
    
    start_time = time.time()
    results_rt = []
    for _ in range(n_repeats):
        if problem == "reg" and model == "dt":
            model_rt = DecisionTree.train_reg(df_train_rt, max_depth=max_depth)
        elif problem == "reg" and model == "rf":
            model_rt = RandomForest.train_reg(df_train_rt, n_estimators=n_estimators, max_depth=max_depth)
        elif problem == "clf" and model == "dt":
            model_rt = DecisionTree.train_clf(df_train_rt, max_depth=max_depth)   
        elif problem == "clf" and model == "rf":
            model_rt = RandomForest.train_clf(df_train_rt, n_estimators=n_estimators, max_depth=max_depth)

        if problem == "reg":
            pred_rt = model_rt.predict(df_test_rt)
            results_rt.append(metric_fn(df_test.target, pred_rt))
        elif problem == "clf":
            pred_rt = model_rt.predict(df_test_rt)
            results_rt.append(metric_fn(df_test.target, np.array(pred_rt) > 0.5))
    rt_time = (time.time() - start_time)/n_repeats
    rt_mean = np.mean(results_rt)
    rt_std = np.std(results_rt)
        
    return (dataset, sk_mean, rt_mean, sk_std, rt_std, sk_time, rt_time, metric)

In [4]:
results_reg = [evaluate_dataset(d, "reg") for d in datasets["reg"]]
results_clf = [evaluate_dataset(d, "clf") for d in datasets["clf"]]
results = results_reg + results_clf

cols = "dataset sk_mean rt_mean sk_std rt_std sk_time(s) rt_time(s) metric".split()

pd.DataFrame(results, columns=cols)


,dataset,sk_mean,rt_mean,sk_std,rt_std,sk_time(s),rt_time(s),metric
0,diabetes,0.302524,0.304619,3.558117e-02,3.931226e-02,0.002442,0.002006,r2
1,housing,0.599732,0.599732,1.236292e-16,1.110223e-16,0.043225,0.067254,r2
2,dgp,0.993509,0.993510,4.440892e-16,0.000000e+00,0.057512,0.454802,r2
3,breast_cancer,0.929649,0.929649,6.664358e-03,6.918156e-03,0.004296,0.004188,acc
4,titanic,0.786441,0.779661,1.110223e-16,3.330669e-16,0.002328,0.002098,acc


In [5]:
results_reg = [evaluate_dataset(d, "reg", model="rf", n_repeats=10) for d in datasets["reg"]]
results_clf = [evaluate_dataset(d, "clf", model="rf", n_repeats=10) for d in datasets["clf"]]
results = results_reg + results_clf

cols = "dataset sk_mean rt_mean sk_std rt_std sk_time(s) rt_time(s) metric".split()

pd.DataFrame(results, columns=cols)


,dataset,sk_mean,rt_mean,sk_std,rt_std,sk_time(s),rt_time(s),metric
0,diabetes,0.422794,0.444132,0.038167,0.016592,0.034998,0.005205,r2
1,housing,0.643550,0.642039,0.002724,0.003161,0.094534,0.187938,r2
2,dgp,0.995655,0.995620,0.000232,0.000152,0.115161,1.895788,r2
3,breast_cancer,0.958246,0.943509,0.012791,0.010809,0.032891,0.016874,acc
4,titanic,0.799322,0.793559,0.007993,0.008502,0.031974,0.006837,acc
